# SQLAlchemy

SQLAlchemy provides an Object Relational Mapper (ORM) that allows us to interact with databases using pythonic object-oriented code rather than writing SQL queries. Out of the box it only works with SQLite, which is based on providing the entire database in a single file and only allows access via a single connection at a time. It is thus not recommended to use SQLite for any production web-based applications or any larger projects. For those cases we will cover PostreSQL in the next lecture.

We use the [Chinook](https://github.com/lerocha/chinook-database) database as our example. The Chinook data model represents a digital media store, including tables for artists, albums, media tracks, invoices and customers.

In [1]:
import sqlalchemy as db

In [4]:
engine = db.create_engine('sqlite:///chinook.db')
connection = engine.connect()
metadata = db.MetaData()
employees = db.Table('employees', metadata, autoload=True, autoload_with=engine)

Now let's see what some common SQL queries look like:

SQL:

- SELECT * FROM employees

In [8]:
query = db.select([employees])
ResultProxy = connection.execute(query)
ResultSet = ResultProxy.fetchall()
ResultSet[:3]

[(1, 'Adams', 'Andrew', 'General Manager', None, datetime.datetime(1962, 2, 18, 0, 0), datetime.datetime(2002, 8, 14, 0, 0), '11120 Jasper Ave NW', 'Edmonton', 'AB', 'Canada', 'T5K 2N1', '+1 (780) 428-9482', '+1 (780) 428-3457', 'andrew@chinookcorp.com'),
 (2, 'Edwards', 'Nancy', 'Sales Manager', 1, datetime.datetime(1958, 12, 8, 0, 0), datetime.datetime(2002, 5, 1, 0, 0), '825 8 Ave SW', 'Calgary', 'AB', 'Canada', 'T2P 2T3', '+1 (403) 262-3443', '+1 (403) 262-3322', 'nancy@chinookcorp.com'),
 (3, 'Peacock', 'Jane', 'Sales Support Agent', 2, datetime.datetime(1973, 8, 29, 0, 0), datetime.datetime(2002, 4, 1, 0, 0), '1111 6 Ave SW', 'Calgary', 'AB', 'Canada', 'T2P 5M5', '+1 (403) 262-3443', '+1 (403) 262-6712', 'jane@chinookcorp.com')]

In [9]:
# Use fetchmany to fetch a specific number of entries (useful for large db)
ResultProxy = connection.execute(query)
ResultSet = ResultProxy.fetchmany(3)
ResultSet

[(1, 'Adams', 'Andrew', 'General Manager', None, datetime.datetime(1962, 2, 18, 0, 0), datetime.datetime(2002, 8, 14, 0, 0), '11120 Jasper Ave NW', 'Edmonton', 'AB', 'Canada', 'T5K 2N1', '+1 (780) 428-9482', '+1 (780) 428-3457', 'andrew@chinookcorp.com'),
 (2, 'Edwards', 'Nancy', 'Sales Manager', 1, datetime.datetime(1958, 12, 8, 0, 0), datetime.datetime(2002, 5, 1, 0, 0), '825 8 Ave SW', 'Calgary', 'AB', 'Canada', 'T2P 2T3', '+1 (403) 262-3443', '+1 (403) 262-3322', 'nancy@chinookcorp.com'),
 (3, 'Peacock', 'Jane', 'Sales Support Agent', 2, datetime.datetime(1973, 8, 29, 0, 0), datetime.datetime(2002, 4, 1, 0, 0), '1111 6 Ave SW', 'Calgary', 'AB', 'Canada', 'T2P 5M5', '+1 (403) 262-3443', '+1 (403) 262-6712', 'jane@chinookcorp.com')]

### Filtering

To filter the results let's see which columns we have available in the table 'employees'

In [10]:
query.columns.keys()

['EmployeeId',
 'LastName',
 'FirstName',
 'Title',
 'ReportsTo',
 'BirthDate',
 'HireDate',
 'Address',
 'City',
 'State',
 'Country',
 'PostalCode',
 'Phone',
 'Fax',
 'Email']

To find all cities in the database:

SQL:

- SELECT City FROM employees

In [11]:
query = db.select([employees.columns.City])
ResultProxy = connection.execute(query)
ResultSet = ResultProxy.fetchall()
ResultSet

[('Edmonton',),
 ('Calgary',),
 ('Calgary',),
 ('Calgary',),
 ('Calgary',),
 ('Calgary',),
 ('Lethbridge',),
 ('Lethbridge',)]

SQL:

- SELECT * FROM employees WHERE City = 'Lethbridge';

In [12]:
query = db.select([employees]).where(employees.columns.City == 'Lethbridge')
ResultProxy = connection.execute(query)
ResultSet = ResultProxy.fetchall()
ResultSet

[(7, 'King', 'Robert', 'IT Staff', 6, datetime.datetime(1970, 5, 29, 0, 0), datetime.datetime(2004, 1, 2, 0, 0), '590 Columbia Boulevard West', 'Lethbridge', 'AB', 'Canada', 'T1K 5N8', '+1 (403) 456-9986', '+1 (403) 456-8485', 'robert@chinookcorp.com'),
 (8, 'Callahan', 'Laura', 'IT Staff', 6, datetime.datetime(1968, 1, 9, 0, 0), datetime.datetime(2004, 3, 4, 0, 0), '923 7 ST NW', 'Lethbridge', 'AB', 'Canada', 'T1H 1Y8', '+1 (403) 467-3351', '+1 (403) 467-8772', 'laura@chinookcorp.com')]

SQL:

- SELECT Title, ReportsTo FROM employees WHERE Title IN ('Sales Manager', 'IT Staff');

In [14]:
query = db.select([employees.columns.Title, employees.columns.ReportsTo]).where(employees.columns.Title.in_(['Sales Manager', 'IT Staff']))
ResultProxy = connection.execute(query)
ResultSet = ResultProxy.fetchall()
ResultSet

[('Sales Manager', 1), ('IT Staff', 6), ('IT Staff', 6)]